<a href="https://colab.research.google.com/github/seijimorimoto/CE888-Data-Science/blob/master/Lab5/CE888_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Lab 5. Recommender Systems**

**1. Loading the *jester-data-1* dataset.**

In [0]:
import pandas as pd
import numpy as np

In [0]:
col_names = ['JokeCount'] + [f'Joke{num + 1}' for num in range(100)]
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',header=None, names=col_names)

In [0]:
df = df.drop(columns=['JokeCount'])

In [0]:
arr = df.values

**2. Labeling 10% of the dataset cells as NaNs (represented as 99 in our case) to create a validation set.**

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, validation_indices = replace(arr, 0.1)

**3. Infer hidden ratings of the users and hidden characteristics of the jokes by utilizing latent factors modeling.**

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [0]:
sgd()

25.053632117133635
22.901327893010883
21.289883849050668
20.29711679089692
18.495648917778848


**4. Comparing predictions (obtained through latent factors modeling) with true values (original dataset).**

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = [f'Joke{num + 1}' for num in range(100)]

In [0]:
comparison_data

,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,Joke40,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,"(99.0, -3.1883958892690005)","(99.0, -3.764506638261345)","(99.0, -4.820591478704082)","(-8.16, -8.385132141409427)","(-7.52, -2.5084620862662903)","(-8.5, -1.5751803876805703)","(-9.85, -4.568982547222585)","(4.17, -3.783547958829055)","(99.0, -7.3431645777557)","(-4.76, -2.9910108061484566)","(-8.5, -1.3716799884801405)","(-6.75, -1.545340877585691)","(-7.18, -5.967084055011345)","(8.45, -1.472993808324943)","(-7.18, -5.8833170875497185)","(-7.52, -9.17504308137995)","(-7.43, -4.092418817164939)","(-9.81, -4.565710249305481)","(-9.85, -3.460714761540243)","(-9.85, -5.645610141865154)","(-9.37, 0.3160672048576742)","(1.5, -3.7131964580413115)","(-4.37, -5.688849826009972)","(-9.81, -9.2544556636687)","(99.0, -4.3158568961549095)","(1.12, -1.5047874963854457)","(7.82, 3.2387251906967425)","(2.86, -1.6846678816614893)","(9.13, 3.3946874769912125)","(-7.43, -7.138840346995856)","(2.14, 1.108237986878809)","(-4.08, 2.9787481023333444)","(-9.08, -8.476918411268972)","(7.82, -2.6192872308375583)","(5.05, 2.7354227025380435)","(4.95, 3.6267242510120807)","(-9.17, -8.074543560285829)","(-8.4, -2.9520411387216168)","(99.0, -2.4483385092662595)","(-8.4, -3.299079838153281)",...,"(8.59, 1.1210292683534746)","(3.59, 1.9395308014378039)","(-6.84, -4.947252640653697)","(-9.03, -7.325446524038105)","(2.82, 1.3677528281596953)","(-1.36, 1.434114377969336)","(-9.08, -7.799092121451946)","(8.3, 1.4075562857074677)","(99.0, 1.687991448650621)","(-4.81, -4.360541048634479)","(99.0, -5.33875730106742)","(99.0, 0.9154413039119257)","(99.0, -2.4650081662539605)","(99.0, -7.098591216439074)","(99.0, -4.797789930963332)","(99.0, 0.5866458451590482)","(99.0, -3.225099349443256)","(-9.42, -1.6276915205570497)","(99.0, -3.973846732792764)","(99.0, -1.9116052314776721)","(99.0, -0.6398598294165625)","(-7.72, -3.5196210960443373)","(99.0, -0.015589205059639974)","(99.0, -3.6070709640361587)","(99.0, -2.8627608965341813)","(99.0, -4.561016094071904)","(99.0, -0.36356520737630565)","(99.0, -0.6973358379246835)","(99.0, 2.759521471523628)","(99.0, -4.104165597603101)","(2.82, -0.8045321262930739)","(99.0, -2.2227276754881338)","(99.0, 0.3066215383184227)","(99.0, -3.309791737789375)","(99.0, -3.0332324189597943)","(99.0, -2.0725992906259134)","(-5.63, -1.5576108984202672)","(99.0, -1.9348708738236442)","(99.0, -4.944718352261926)","(99.0, -1.7075386589627728)"
1,"(4.08, 2.8831219765214375)","(-0.29, 3.2038812798831757)","(6.36, 3.615952078382612)","(4.37, 4.772747477519686)","(-2.38, 2.3318571392772713)","(-9.66, 2.820866687300003)","(-0.73, 2.8759381610871504)","(-5.34, 2.079253312202786)","(8.88, 4.4702611875595375)","(9.22, 2.9289636396293814)","(6.75, 2.6867121424280076)","(8.64, 2.88007372881708)","(4.42, 2.7943032434583523)","(7.43, 2.8480192430193245)","(99.0, 2.5610390479450715)","(-0.97, 3.6562957949216135)","(4.66, 2.2751889286264926)","(99.0, 2.6373827609834866)","(3.3, 2.719425794484822)","(-1.21, 3.4253741771369084)","(0.87, 2.1420799855130377)","(8.64, 3.5143175522782375)","(8.35, 4.047643942420594)","(9.17, 4.891088734680343)","(0.05, 3.660047271563889)","(7.57, 2.799991660995579)","(99.0, 0.8758525120098716)","(0.87, 2.7925808580413767)","(-0.39, 0.6789812116090436)","(6.99, 4.6709445477910485)","(99.0, 1.7387573577790794)","(-0.92, 0.9859334252894942)","(7.14, 4.816389265785961)","(9.03, 3.082197876504474)","(99.0, 1.1049999219886675)","(0.73, 0.6025807183177946)","(7.09, 4.360475864818317)",

**5. Calculating the performance (MSE) of the trained latent factors model on the validation set.**

In [28]:
original_data = arr
assert len(validation_indices[0]) == len(validation_indices[1])
validation_size = len(validation_indices[0])
errors = []
for i in range(validation_size):
  user_id = validation_indices[0][i]
  joke_id = validation_indices[1][i]
  prediction_rating = predict_rating(user_id, joke_id)
  real_rating = original_data[user_id][joke_id]
  error = prediction_rating - real_rating
  errors.append(error)
mse = (np.array(errors) ** 2).mean()
print('Validation MSE:', mse)

Validation MSE: 18.304230220838267


**6. Making predictions on the test set.**

In [0]:
filled_data = np.zeros(original_data.shape)
rows, columns = original_data.shape
for user_id in range(rows):
  for joke_id in range(columns):
    if original_data[user_id][joke_id] == 99:
      filled_data[user_id][joke_id] = predict_rating(user_id, joke_id)
    else:
      filled_data[user_id][joke_id] = original_data[user_id][joke_id]

In [0]:
filled_data

array([[-7.82      ,  8.79      , -9.66      , ..., -1.93487087,
        -4.94471835, -1.70753866],
       [ 4.08      , -0.29      ,  6.36      , ...,  0.34      ,
        -4.32      ,  1.07      ],
       [ 5.82824374,  6.16250978,  6.14356943, ...,  6.38359923,
         5.68083207,  7.27976696],
       ...,
       [-0.23012087, -0.26492064, -0.32275439, ..., -0.16650137,
        -0.32520792, -0.16246711],
       [ 0.27090079,  0.23242422,  0.08506485, ...,  0.51061821,
         0.01173731,  0.65083534],
       [ 2.43      ,  2.67      , -3.98      , ...,  3.4894801 ,
         2.32802986,  4.09969455]])

**7. Finding the best and worst rated jokes.**

In [0]:
means = np.mean(filled_data, axis=0)

In [0]:
max_joke = -9999999999
max_joke_index = -1
min_joke = 99999999999
min_joke_index = -1

for i in range(len(means)):
  if means[i] > max_joke:
    max_joke = means[i]
    max_joke_index = i
  elif means[i] < min_joke:
    min_joke = means[i]
    min_joke_index = i

print(f'The best-rated joke was joke {max_joke_index + 1}, with a mean rating of: {max_joke}')
print(f'The worst-rated joke was joke {min_joke_index + 1}, with a mean rating of: {min_joke}')

The best-rated joke was joke 50, with a mean rating of: 3.6655005509779905
The worst-rated joke was joke 58, with a mean rating of: -3.343396876647032
